In [7]:
# If needed (uncomment):
# %pip install ultralytics opencv-python numpy pandas pyyaml

import os, math, yaml, cv2, pathlib, random
import numpy as np
import pandas as pd
from collections import defaultdict
from datetime import datetime
from ultralytics import YOLO

print("OpenCV:", cv2.__version__)


OpenCV: 4.12.0


In [8]:
# 🔁 Set your own paths here
VIDEO_PATH  = './input.mp4'            # <-- change to your actual video
CFG_PATH    = './roi_config.yaml'      # will be created below if missing
OUT_VIDEO   = './output_footfall.mp4'
EVENTS_CSV  = './events_humans.csv'
SUMMARY_CSV = './summary_humans.csv'
SNAPS_DIR   = './events_snaps'

# One-time: create a default roi_config.yaml if it doesn't exist
if not os.path.exists(CFG_PATH):
    default_cfg = {
        'global': {
            'model_weights': 'yolov8n.pt',
            'resize_width': 960,           # None to keep original width
            'conf_thresh': 0.35,
            'iou_thresh': 0.45,
            'min_confidence': 0.35,
            'process_every_nth_frame': 1,  # >1 to skip frames for speed
            'min_track_age': 2,
            'cooldown_frames': 18,
            'min_normal_pixels': 18,
            'hysteresis_margin_pct': 25
        },
        'rois': {
            'N':  {'height_pct': 8},
            'NE': {'height_pct': 8},
            'E':  {'height_pct': 8},
            'SE': {'height_pct': 8},
            'S':  {'height_pct': 8},
            'SW': {'height_pct': 8},
            'W':  {'height_pct': 8},
            'NW': {'height_pct': 8}
        }
    }
    with open(CFG_PATH, 'w', encoding='utf-8') as f:
        yaml.safe_dump(default_cfg, f, sort_keys=False)
    print("Created default roi_config.yaml")
else:
    print("Using existing roi_config.yaml")


Using existing roi_config.yaml


In [9]:
# -----------------------------
# Robust YAML loader (Windows)
# -----------------------------
def load_cfg(path='./roi_config.yaml'):
    encodings_to_try = ['utf-8', 'utf-8-sig', 'cp1252', 'latin-1']
    last_err = None
    for enc in encodings_to_try:
        try:
            with open(path, 'r', encoding=enc, errors='strict') as f:
                text = f.read()
            text = (text
                    .replace('\u201c', '"').replace('\u201d', '"')
                    .replace('\u2018', "'").replace('\u2019', "'")
                    .replace('\u2013', '-').replace('\u2014', '-'))
            return yaml.safe_load(text)
        except Exception as e:
            last_err = e
    with open(path, 'rb') as f:
        raw = f.read()
    text = raw.decode('utf-8', errors='ignore')
    try:
        return yaml.safe_load(text)
    except Exception as e:
        print(f"[ERROR] Could not load YAML even after fallback: {e}")
        raise last_err

# -------------------------------
# Path-safe video opener + probe
# -------------------------------
def open_video(path):
    p = str(pathlib.Path(path).expanduser().resolve())
    cap = cv2.VideoCapture(p, cv2.CAP_FFMPEG)
    if not cap.isOpened():
        cap = cv2.VideoCapture(p)
    if not cap.isOpened():
        raise FileNotFoundError(f"Cannot open video at: {p}")
    return cap

def probe_video(cap, label):
    W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    FPS = cap.get(cv2.CAP_PROP_FPS) or 30.0
    N = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if W == 0 or H == 0:
        raise RuntimeError(f"{label}: video opened but has 0x0 size — likely unsupported codec. Re-encode to H.264 MP4.")
    print(f"[Video] {label} | {W}x{H} | {FPS:.2f} FPS | {N} frames")
    return W, H, FPS, N

# -------------------------------
# Direction & band helpers
# -------------------------------
def angle_to_dir(angle_deg):
    ang = (angle_deg + 360.0) % 360.0
    bins = [(0,'E'), (45,'NE'), (90,'N'), (135,'NW'), (180,'W'), (225,'SW'), (270,'S'), (315,'SE'), (360,'E')]
    for i in range(len(bins)-1):
        if bins[i][0] <= ang < bins[i+1][0]:
            return bins[i][1]
    return 'E'

def build_auto_band(direction, W, H):
    if direction in ['N','S']:
        y = H // 2
        return [(0, y), (W-1, y)]
    if direction in ['E','W']:
        x = W // 2
        return [(x, 0), (x, H-1)]
    if direction in ['NE','SW']:
        return [(0, H-1), (W-1, 0)]
    if direction in ['NW','SE']:
        return [(W-1, H-1), (0, 0)]
    return [(0, H//2), (W-1, H//2)]

def band_to_rects(centerline, height, margin, W, H):
    (x1,y1),(x2,y2) = centerline
    dx, dy = (x2-x1), (y2-y1)
    L = math.hypot(dx, dy) + 1e-6
    nx, ny = -dy/L, dx/L  # unit normal (perp)
    half = height/2
    inner_half = max(1, half - margin)
    outer_half = half + margin

    def clip(x,y):
        return int(np.clip(x, 0, W-1)), int(np.clip(y, 0, H-1))

    pts_inner = np.array([
        clip(x1 + nx*inner_half, y1 + ny*inner_half),
        clip(x2 + nx*inner_half, y2 + ny*inner_half),
        clip(x2 - nx*inner_half, y2 - ny*inner_half),
        clip(x1 - nx*inner_half, y1 - ny*inner_half),
    ])
    pts_outer = np.array([
        clip(x1 + nx*outer_half, y1 + ny*outer_half),
        clip(x2 + nx*outer_half, y2 + ny*outer_half),
        clip(x2 - nx*outer_half, y2 - ny*outer_half),
        clip(x1 - nx*outer_half, y1 - ny*outer_half),
    ])

    ix1, iy1 = pts_inner.min(axis=0); ix2, iy2 = pts_inner.max(axis=0)
    ox1, oy1 = pts_outer.min(axis=0); ox2, oy2 = pts_outer.max(axis=0)

    inner_rect = (int(ix1), int(iy1), int(ix2), int(iy2))
    outer_rect = (int(ox1), int(oy1), int(ox2), int(oy2))
    return inner_rect, outer_rect, (nx, ny)

def point_in_rect(pt, rect):
    x,y = pt; x1,y1,x2,y2 = rect
    return (x1 <= x <= x2) and (y1 <= y <= y2)

def resolve_height(roi_entry, W, H):
    if isinstance(roi_entry.get('height', None), (int, float)):
        return int(roi_entry['height'])
    hpct = roi_entry.get('height_pct', None)
    if hpct is None:
        hpct = 8
    return max(2, int(round((float(hpct) / 100.0) * min(W, H))))

def resolve_margin(global_cfg, band_height):
    if isinstance(global_cfg.get('hysteresis_margin', None), (int, float)):
        return int(global_cfg['hysteresis_margin'])
    mpct = global_cfg.get('hysteresis_margin_pct', 25)
    return max(1, int(round((float(mpct) / 100.0) * band_height)))

def project_to_segment(px, py, x1, y1, x2, y2):
    dx, dy = (x2 - x1), (y2 - y1)
    L2 = dx*dx + dy*dy + 1e-9
    t = ((px - x1)*dx + (py - y1)*dy) / L2
    t_clamped = max(0.0, min(1.0, t))
    projx = x1 + t_clamped * dx
    projy = y1 + t_clamped * dy
    return projx, projy, t_clamped

def draw_segment_hist(frame, hist_in, hist_out, x=20, y=140, w=300, h=60):
    bins = len(hist_in)
    if bins == 0: return
    bw = max(1, w // bins)
    maxv = max(1, max(hist_in + hist_out))
    # IN top half (green), OUT bottom half (yellow-ish)
    for i,v in enumerate(hist_in):
        bh = int((h//2) * (v / maxv))
        cv2.rectangle(frame, (x + i*bw, y + (h//2 - bh)), (x + (i+1)*bw - 2, y + h//2 - 2), (0,180,0), -1)
    for i,v in enumerate(hist_out):
        bh = int((h//2) * (v / maxv))
        cv2.rectangle(frame, (x + i*bw, y + h//2 + 2), (x + (i+1)*bw - 2, y + h//2 + 2 + bh), (0,255,255), -1)
    cv2.rectangle(frame, (x, y), (x+w, y+h), (255,255,255), 1)
    cv2.putText(frame, "ROI segment histogram  (IN top, OUT bottom)", (x, y-6),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200,200,200), 1)
    
# -------- Single-line ROI helpers --------
def dir_unit(label):
    vecs = {
        'E': (1,0), 'NE': (1,-1), 'N': (0,-1), 'NW': (-1,-1),
        'W': (-1,0), 'SW': (-1,1), 'S': (0,1), 'SE': (1,1)
    }
    vx, vy = vecs.get(label, (1,0))
    L = math.hypot(vx, vy) or 1.0
    return (vx/L, vy/L)

def unit_perp(x1, y1, x2, y2):
    dx, dy = (x2-x1), (y2-y1)
    L = math.hypot(dx, dy) or 1.0
    # perpendicular to line (tangent=[dx,dy]), normal = [-dy, dx]
    return (-dy/L, dx/L)

def compute_aligned_normal(centerline, chosen_dir):
    (x1,y1),(x2,y2) = centerline
    nx, ny = unit_perp(x1,y1,x2,y2)
    ux, uy = dir_unit(chosen_dir)        # desired IN direction
    # flip normal so it points roughly along desired flow
    if nx*ux + ny*uy < 0:
        nx, ny = -nx, -ny
    return (nx, ny)

def signed_distance_to_line(px, py, x1, y1, nx, ny):
    # signed distance of point P to infinite line passing through (x1,y1) with normal n
    return (px - x1)*nx + (py - y1)*ny

def draw_centerline(frame, centerline, color=(255,255,0), thickness=2):
    (x1,y1),(x2,y2) = centerline
    cv2.line(frame, (int(x1),int(y1)), (int(x2),int(y2)), color, thickness)

def project_to_segment(px, py, x1, y1, x2, y2):
    dx, dy = (x2 - x1), (y2 - y1)
    L2 = dx*dx + dy*dy + 1e-9
    t = ((px - x1)*dx + (py - y1)*dy) / L2
    t_clamped = max(0.0, min(1.0, t))
    projx = x1 + t_clamped * dx
    projy = y1 + t_clamped * dy
    return projx, projy, t_clamped
# -------- 8-dir single-line helpers --------
DIRECTIONS = ['N','NE','E','SE','S','SW','W','NW']

def dir_unit(label):
    vecs = {
        'E': (1,0), 'NE': (1,-1), 'N': (0,-1), 'NW': (-1,-1),
        'W': (-1,0), 'SW': (-1,1), 'S': (0,1), 'SE': (1,1)
    }
    vx, vy = vecs.get(label, (1,0))
    L = math.hypot(vx, vy) or 1.0
    return (vx/L, vy/L)

def unit_perp(x1, y1, x2, y2):
    dx, dy = (x2-x1), (y2-y1)
    L = math.hypot(dx, dy) or 1.0
    # normal perpendicular to line (tangent=[dx,dy]) is [-dy, dx]
    return (-dy/L, dx/L)

def compute_aligned_normal(centerline, chosen_dir):
    (x1,y1),(x2,y2) = centerline
    nx, ny = unit_perp(x1,y1,x2,y2)
    ux, uy = dir_unit(chosen_dir)        # intended "IN" direction
    # flip normal so +normal points along desired flow
    if nx*ux + ny*uy < 0:
        nx, ny = -nx, -ny
    return (nx, ny)

def signed_distance_to_line(px, py, x1, y1, nx, ny):
    # signed distance of point P to infinite line through (x1,y1) with normal n
    return (px - x1)*nx + (py - y1)*ny

def draw_centerline(frame, centerline, color=(180,180,180), thickness=1):
    (x1,y1),(x2,y2) = centerline
    cv2.line(frame, (int(x1),int(y1)), (int(x2),int(y2)), color, thickness)

def project_to_segment(px, py, x1, y1, x2, y2):
    dx, dy = (x2 - x1), (y2 - y1)
    L2 = dx*dx + dy*dy + 1e-9
    t = ((px - x1)*dx + (py - y1)*dy) / L2
    t_clamped = max(0.0, min(1.0, t))
    projx = x1 + t_clamped * dx
    projy = y1 + t_clamped * dy
    return projx, projy, t_clamped




In [10]:
# -------------------------------
# Flow estimation (first seconds)
# -------------------------------
def estimate_dominant_direction(video_path, model, secs=6, stride=2, resize_w=None, conf=0.35, iou=0.45):
    cap = open_video(video_path)
    W,H,FPS,N = probe_video(cap, "FlowProbe")
    n_frames = int(secs * FPS)
    prev_centers = None
    hist = defaultdict(int)
    f = 0
    while True:
        ok, frame = cap.read()
        if not ok: break
        f += 1
        if f > n_frames: break
        if f % stride != 0: continue

        proc = frame
        if resize_w and frame.shape[1] > resize_w:
            s = resize_w / frame.shape[1]
            proc = cv2.resize(frame, (resize_w, int(frame.shape[0]*s)))
        else:
            s = 1.0

        res = model.predict(proc, classes=[0], conf=conf, iou=iou, verbose=False, device='cpu')
        sx = frame.shape[1] / proc.shape[1]; sy = frame.shape[0] / proc.shape[0]
        centers = []
        if res and res[0].boxes is not None:
            for b in res[0].boxes.xyxy.cpu().numpy():
                x1,y1,x2,y2 = b
                centers.append((int((x1+x2)/2*sx), int((y1+y2)/2*sy)))

        if prev_centers:
            used = set()
            for cx,cy in centers:
                best = None; bestd = 1e9; bestj = -1
                for j,(px,py) in enumerate(prev_centers):
                    if j in used: continue
                    d = (cx-px)**2 + (cy-py)**2
                    if d < bestd:
                        bestd = d; best = (px,py); bestj = j
                if best is not None and bestd < 80**2:
                    dx,dy = cx-best[0], cy-best[1]
                    if abs(dx)+abs(dy) >= 4:
                        ang = math.degrees(math.atan2(-dy, dx))
                        hist[angle_to_dir(ang)] += 1
                    used.add(bestj)
        prev_centers = centers
    cap.release()
    if not hist:
        return 'N', (W,H), dict(hist)
    chosen = max(hist.items(), key=lambda kv: kv[1])[0]
    return chosen, (W,H), dict(hist)

# -------------------------------
# Custom centroid tracker with fade
# -------------------------------
class CentroidTracker:
    def __init__(self, max_link=60):
        self.max_link = max_link
        self.next_id = 1
        self.tracks = {}
    def update(self, centers, bboxes, frame_idx):
        assigned = set()
        new_tracks = {}
        for tid,data in self.tracks.items():
            px,py = data['center']
            bestj, bestd = -1, 1e9
            for j,(cx,cy) in enumerate(centers):
                if j in assigned: continue
                d = (cx-px)**2 + (cy-py)**2
                if d < bestd:
                    bestj, bestd = j, d
            if bestj >= 0 and bestd <= self.max_link**2:
                cx,cy = centers[bestj]; assigned.add(bestj)
                bb = bboxes[bestj] if bestj < len(bboxes) else data.get('bbox', (cx-10,cy-10,cx+10,cy+10))
                new_tracks[tid] = {
                    'center': (cx,cy),
                    'bbox': bb,
                    'age': data['age'] + 1,
                    'last_seen': frame_idx
                }
        for j,(cx,cy) in enumerate(centers):
            if j in assigned: continue
            tid = self.next_id; self.next_id += 1
            bb = bboxes[j] if j < len(bboxes) else (cx-10,cy-10,cx+10,cy+10)
            new_tracks[tid] = {
                'center': (cx,cy),
                'bbox': bb,
                'age': 1,
                'last_seen': frame_idx
            }
        self.tracks = new_tracks
        return self.tracks

def id_color(tid):
    random.seed(int(tid) * 111)
    r = random.randint(80,255)
    g = random.randint(80,255)
    b = random.randint(80,255)
    return (b,g,r)

def draw_boxes_with_fade(frame, active_tracks, fade_mem, frame_idx, fade_frames=20):
    """Draw regular boxes and faded memory boxes."""
    overlay = frame.copy()
    active_tids = set(active_tracks.keys())
    # Active
    for tid, data in active_tracks.items():
        cx, cy = data['center']
        X1,Y1,X2,Y2 = data['bbox']
        col = id_color(tid)
        cv2.rectangle(overlay, (X1,Y1), (X2,Y2), col, 2)
        cv2.circle(overlay, (cx,cy), 3, col, -1)
        cv2.putText(overlay, f"ID {tid}", (X1+4, max(15,Y1-6)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, col, 2)
        fade_mem[tid] = {'bbox': (X1,Y1,X2,Y2), 'color': col, 'last_seen': frame_idx}
    # Faded
    for tid, mem in list(fade_mem.items()):
        if tid in active_tids:
            continue
        age_off = frame_idx - mem['last_seen']
        if age_off <= fade_frames:
            alpha = max(0.0, 1.0 - (age_off / float(fade_frames)))
            X1,Y1,X2,Y2 = mem['bbox']
            col = mem['color']
            faded = frame.copy()
            cv2.rectangle(faded, (X1,Y1), (X2,Y2), col, 2)
            frame[:] = cv2.addWeighted(faded, alpha, frame, 1.0 - alpha, 0)
        else:
            fade_mem.pop(tid, None)
    frame[:] = overlay

# ✅ Event highlighting ring (GREEN for IN, YELLOW for OUT)
def draw_event_ring(frame, cx, cy, kind):
    color = (0,255,0) if kind == 'IN' else (0,255,255)  # GREEN / YELLOW
    cv2.circle(frame, (int(cx), int(cy)), 22, color, 3)
    cv2.putText(frame, kind, (int(cx)-18, int(cy)-28),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)


In [ ]:
def run(video_path, cfg_path, out_video, events_csv, summary_csv):
    cfg = load_cfg(cfg_path)
    g = cfg['global']; rois_cfg = cfg['rois']
    model = YOLO(g['model_weights']).to('cpu')

    # 1) Probe dominant flow (for HUD & normal alignment defaults)
    chosen_flow, (W,H), hist = estimate_dominant_direction(
        video_path, model,
        secs=6, stride=2, resize_w=g['resize_width'],
        conf=g['conf_thresh'], iou=g['iou_thresh']
    )
    print(f"[Auto] Dominant flow ≈ {chosen_flow}  (hist={hist})")

    # 2) Build/resolve SINGLE LINE per each of 8 directions + aligned normal
    DIRECTIONS = ['N','NE','E','SE','S','SW','W','NW']
    roi_lines = {}      # dir -> dict(centerline, nx, ny, source)
    for d in DIRECTIONS:
        r = rois_cfg.get(d, {}) if isinstance(rois_cfg, dict) else {}
        if r.get('band', None) is None:
            centerline = build_auto_band(d, W, H)  # auto centerline per dir
            source = 'auto'
        else:
            (x1,y1),(x2,y2) = r['band']
            centerline = [(int(x1),int(y1)), (int(x2),int(y2))]
            source = 'cfg'
        nx, ny = compute_aligned_normal(centerline, d)  # +normal => "towards d"
        roi_lines[d] = {'centerline': centerline, 'nx': nx, 'ny': ny, 'source': source}

    # 3) Counting loop
    cap = open_video(video_path)
    W2,H2,FPS,N = probe_video(cap, "Main")
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(out_video, fourcc, FPS/max(1,g['process_every_nth_frame']), (W2,H2))

    tracker = CentroidTracker(max_link=70)
    frame_idx = 0

    # per-track cooldown & prev signed distance per direction
    last_event_frame = {}                # tid -> frame idx of last accepted event
    from collections import defaultdict as dd
    prev_signed = dd(dict)               # tid -> {dir: s_prev}
    prev_t_along = dd(dict)              # tid -> {dir: t_prev}

    # Global (center) totals
    in_count = out_count = 0
    events = []

    # ✅ Quadrant buckets
    QUADS = ['NE','NW','SE','SW']
    quad_counts = {q: {'IN':0, 'OUT':0} for q in QUADS}

    # thresholds
    min_normal_px = max(3, int(g.get('min_normal_pixels', 10)))
    cooldown = int(g.get('cooldown_frames', 12))

    # vis memory
    fade_memory = {}
    os.makedirs(SNAPS_DIR, exist_ok=True)
    event_flash_until = {}               # tid -> frame until highlight persists
    event_kind_mem = {}                  # tid -> 'IN'/'OUT'
    EVENT_FLASH_FRAMES = 12

    # per-direction histograms (for debugging/HUD)
    BAND_SEGMENTS = 20
    hist_in = {d: [0]*BAND_SEGMENTS for d in DIRECTIONS}
    hist_out = {d: [0]*BAND_SEGMENTS for d in DIRECTIONS}

    while True:
        ok, frame = cap.read()
        if not ok: break
        frame_idx += 1
        if frame_idx % max(1,g['process_every_nth_frame']) != 0:
            out.write(frame); continue

        # Inference (resize if needed)
        proc = frame
        if g['resize_width'] and frame.shape[1] > g['resize_width']:
            s = g['resize_width'] / frame.shape[1]
            proc = cv2.resize(frame, (g['resize_width'], int(frame.shape[0]*s)))
        else:
            s = 1.0

        res = model.predict(proc, classes=[0], conf=g['conf_thresh'], iou=g['iou_thresh'],
                            verbose=False, device='cpu')
        sx, sy = frame.shape[1]/proc.shape[1], frame.shape[0]/proc.shape[0]

        centers, bboxes = [], []
        if res and res[0].boxes is not None:
            data = res[0].boxes.xyxy.cpu().numpy()
            confs = res[0].boxes.conf.cpu().numpy()
            for (bx1,by1,bx2,by2), c in zip(data, confs):
                if c < g['min_confidence']: continue
                X1,Y1,X2,Y2 = int(bx1*sx), int(by1*sy), int(bx2*sx), int(by2*sy)
                cx, cy = (X1+X2)//2, (Y1+Y2)//2
                centers.append((cx,cy))
                bboxes.append((X1,Y1,X2,Y2))

        tracks = tracker.update(centers, bboxes, frame_idx)

        # Draw all 8 lines; highlight dominant flow line
        for d, info in roi_lines.items():
            draw_centerline(frame, info['centerline'],
                            color=(180,180,180) if d != chosen_flow else (255,255,0),
                            thickness=2 if d == chosen_flow else 1)

        # Draw boxes (active + fade)
        draw_boxes_with_fade(frame, tracks, fade_memory, frame_idx, fade_frames=20)

        # --- Collect candidate crossings per track across ALL 8 lines ---
        for tid, data in tracks.items():
            cx, cy = data['center']

            # track-level cooldown
            last_evt = last_event_frame.get(tid, -10**9)
            if (frame_idx - last_evt) < cooldown:
                if event_flash_until.get(tid, -1) >= frame_idx:
                    kind = event_kind_mem.get(tid, 'IN')
                    color = (0,255,0) if kind=='IN' else (0,255,255)
                    cv2.circle(frame, (int(cx), int(cy)), 22, color, 2)
                continue

            candidates = []  # (score, kind, dir, seg_idx, t_now, delta_s)

            for d, info in roi_lines.items():
                (x1,y1),(x2,y2) = info['centerline']
                nx, ny = info['nx'], info['ny']

                s_now = signed_distance_to_line(cx, cy, x1, y1, nx, ny)
                _, _, t_now = project_to_segment(cx, cy, x1, y1, x2, y2)

                # seed previous if missing
                if d not in prev_signed[tid]:
                    prev_signed[tid][d] = s_now
                    prev_t_along[tid][d] = t_now
                    continue

                s_prev = prev_signed[tid][d]
                prev_signed[tid][d] = s_now
                prev_t_along[tid][d] = t_now

                # must cross the segment span (t within [0,1])
                if not (0.0 <= t_now <= 1.0): 
                    continue

                # detect sign change across zero AND enough motion along the normal
                if s_prev == 0: 
                    continue
                if (s_prev < 0 and s_now > 0) or (s_prev > 0 and s_now < 0):
                    delta_s = s_now - s_prev
                    if abs(delta_s) >= min_normal_px:
                        is_in = (delta_s > 0)  # along +normal => IN
                        kind = 'IN' if is_in else 'OUT'
                        score = abs(delta_s)
                        seg_idx = max(1, min(BAND_SEGMENTS, int(t_now * BAND_SEGMENTS) + 1))
                        candidates.append((score, kind, d, seg_idx, t_now, delta_s))

            # Commit at most ONE event (highest score) to avoid double-counts
            if candidates:
                candidates.sort(reverse=True, key=lambda x: x[0])
                score, kind, d, seg_idx, t_now, delta_s = candidates[0]

                # Center totals
                if kind == 'IN': in_count += 1
                else: out_count += 1

                # ✅ Quadrant totals: only for diagonal directions
                if d in quad_counts:
                    quad_counts[d][kind] += 1

                last_event_frame[tid] = frame_idx
                event_flash_until[tid] = frame_idx + EVENT_FLASH_FRAMES
                event_kind_mem[tid] = kind

                # Visual ring at centroid
                color = (0,255,0) if kind=='IN' else (0,255,255)
                cv2.circle(frame, (int(cx), int(cy)), 22, color, 3)
                cv2.putText(frame, kind, (int(cx)-22, int(cy)-26),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

                # update per-direction histogram (optional)
                if kind == 'IN': hist_in[d][seg_idx-1] += 1
                else:            hist_out[d][seg_idx-1] += 1

                # snapshot + log
                snap_path = os.path.join(SNAPS_DIR, f"ev_{len(events)+1:03d}_{kind}_{d}_f{frame_idx}.png")
                cv2.imwrite(snap_path, frame)

                events.append({
                    'timestamp': datetime.now().isoformat(timespec='seconds'),
                    'track_id': tid, 'type': kind, 'roi_dir': d, 'flow_dir': chosen_flow,
                    'frame_idx': frame_idx, 'cx': int(cx), 'cy': int(cy),
                    't_0to1': round(float(t_now), 4), 'segment_idx': int(seg_idx),
                    'segments_total': int(BAND_SEGMENTS),
                    'delta_s': round(float(delta_s), 2),
                    'snap': snap_path
                })

        # linger rings
        for tid, data in tracks.items():
            if event_flash_until.get(tid, -1) >= frame_idx:
                kind = event_kind_mem.get(tid, 'IN')
                color = (0,255,0) if kind=='IN' else (0,255,255)
                cx, cy = data['center']
                cv2.circle(frame, (int(cx), int(cy)), 22, color, 2)

        # HUD: center totals + quadrant mini-counters
        cv2.rectangle(frame, (10,10), (360,95), (0,0,0), -1)
        cv2.putText(frame, f"CENTER IN : {in_count}",  (20, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
        cv2.putText(frame, f"CENTER OUT: {out_count}", (20, 85), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,255), 2)

        # quadrant boxes (corners)
        pad = 12
        # NW (top-left)
        cv2.rectangle(frame, (pad, pad+110), (pad+155, pad+170), (30,30,30), -1)
        cv2.putText(frame, f"NW IN:{quad_counts['NW']['IN']} OUT:{quad_counts['NW']['OUT']}",
                    (pad+8, pad+150), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200,200,200), 1)
        # NE (top-right)
        cv2.rectangle(frame, (W2-155-pad, pad+110), (W2-pad, pad+170), (30,30,30), -1)
        cv2.putText(frame, f"NE IN:{quad_counts['NE']['IN']} OUT:{quad_counts['NE']['OUT']}",
                    (W2-146-pad, pad+150), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200,200,200), 1)
        # SW (bottom-left)
        cv2.rectangle(frame, (pad, H2-60-pad), (pad+155, H2-pad), (30,30,30), -1)
        cv2.putText(frame, f"SW IN:{quad_counts['SW']['IN']} OUT:{quad_counts['SW']['OUT']}",
                    (pad+8, H2-20-pad), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200,200,200), 1)
        # SE (bottom-right)
        cv2.rectangle(frame, (W2-155-pad, H2-60-pad), (W2-pad, H2-pad), (30,30,30), -1)
        cv2.putText(frame, f"SE IN:{quad_counts['SE']['IN']} OUT:{quad_counts['SE']['OUT']}",
                    (W2-146-pad, H2-20-pad), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200,200,200), 1)

        # Optional: histogram for dominant flow line only (to avoid clutter)
        draw_segment_hist(frame, hist_in[chosen_flow], hist_out[chosen_flow], x=20, y=180, w=300, h=60)

        out.write(frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    # Save detailed event log
    if events:
        pd.DataFrame(events).to_csv(events_csv, index=False)

    # Save center totals (compat)
    pd.DataFrame([{
        'CENTER_IN': in_count,
        'CENTER_OUT': out_count,
        'flow_direction': chosen_flow,
        'multi_line': True
    }]).to_csv(summary_csv, index=False)

    # ✅ Save quadrant summary as a separate CSV next to summary_csv
    quadrant_csv = summary_csv.replace('.csv', '_quadrants.csv')
    rows = []
    for q in ['NW','NE','SW','SE']:
        rows.append({'quadrant': q,
                     'IN': quad_counts[q]['IN'],
                     'OUT': quad_counts[q]['OUT']})
    pd.DataFrame(rows).to_csv(quadrant_csv, index=False)

    print('\n✅ Done.')
    print(f'• CENTER: IN={in_count} | OUT={out_count}')
    print(f"• QUADRANTS: NW{quad_counts['NW']}  NE{quad_counts['NE']}  SW{quad_counts['SW']}  SE{quad_counts['SE']}")
    print(f"• Video : {out_video}")
    print(f"• Events: {events_csv}")
    print(f"• Summary (center): {summary_csv}")
    print(f"• Summary (quadrants): {quadrant_csv}")
    print(f"• Snapshots: {SNAPS_DIR}/")


In [12]:
print("🔍 Initializing YOLO and starting Footfall Counter...\n")

run(
    video_path=VIDEO_PATH,
    cfg_path=CFG_PATH,
    out_video=OUT_VIDEO,
    events_csv=EVENTS_CSV,
    summary_csv=SUMMARY_CSV
)

# Show summary & sample events
from IPython.display import display, Video

print("\n📊 Summary Output:")
display(pd.read_csv(SUMMARY_CSV))

if os.path.exists(EVENTS_CSV) and os.path.getsize(EVENTS_CSV) > 0:
    print("\n🧾 First few event logs:")
    display(pd.read_csv(EVENTS_CSV).head())

print("\n🎬 Processed Video Preview:")
if os.path.exists(OUT_VIDEO):
    display(Video(OUT_VIDEO, embed=True, width=640))
else:
    print("Processed video not found:", OUT_VIDEO)


🔍 Initializing YOLO and starting Footfall Counter...

[Video] FlowProbe | 1920x1080 | 25.00 FPS | 341 frames


KeyboardInterrupt: 